### 기존 europe.geojson 파일에서 프로젝트에 필요한 20개국의 geojson 파일 생성

In [123]:
import json
import pandas as pd

with open("countryURL.json", "r") as f:
    urls = json.load(f)
country = list(urls.keys())
country.append("United Kingdom")
country.append("Czech Republic")

europe = pd.read_json("europe.geojson")
geojson = dict({"type": "FeatureCollection"})
l = []

for i in range(len(europe)):
    if europe["features"][i]["properties"]["NAME"] in country:
        l.append(europe["features"][i])

geojson["features"] = l

with open("geo.geojson", "w") as f:
    json.dump(geojson, f)

### Folium

In [61]:
import folium
import pandas as pd

data = pd.read_csv("./DS/score.csv")
data.loc[3, "country"]="Czech Republic"
data.loc[18, "country"]="United Kingdom"

geo_data = 'geo.geojson'

center = [55.4700, 11.9175]

map = folium.Map(location=center, zoom_start=3, tooltip = 'This tooltip will appear on hover')

folium.Choropleth(
    geo_data=geo_data,
    data=data,
    columns=('country','vader_com'),
    key_on='feature.properties.NAME',
    fill_color='YlGn'
).add_to(map)

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.1, 
                                'weight': 1}
NIL = folium.features.GeoJson(
    geo_data,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    
    tooltip=folium.features.GeoJsonTooltip(
        fields=['NAME'],
        aliases=['']
    )
)

map.add_child(NIL)
map.keep_in_front(NIL)
folium.LayerControl().add_to(map)

folium.features.ClickForMarker().add_to(map)

map